In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

ImportError: Could not find 'cudnn64_8.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Note that installing cuDNN is a separate step from installing CUDA, and this DLL is often found in a different directory from the CUDA DLLs. You may install the necessary DLL by downloading cuDNN 7 from this URL: https://developer.nvidia.com/cudnn

Parameters

In [ ]:
num_classes = 10
min_width = 224
min_height = 224
batch_size=8

data preparations

In [ ]:
from pathlib import Path
import pandas as pd

training_data = Path('../input/10-monkey-species/training/training/') 
validation_data = Path('../input/10-monkey-species/validation/validation/') 
labels_path = Path('../input/10-monkey-species/monkey_labels.txt')


labels_info = []

# Read and cleanup the file
lines = labels_path.read_text().strip().splitlines()
for line in lines[1:]:
    line = line.split(',')
    line = [x.strip(' \n\t\r') for x in line]
    line[3], line[4] = int(line[3]), int(line[4])
    line = tuple(line)
    labels_info.append(line)
    
# Convert the data into a pandas dataframe
labels_info = pd.DataFrame(labels_info, columns=['Label', 'Latin Name', 'Common Name', 
                                                 'Train Images', 'Validation Images'], index=None)
# Show Labels
labels_info.head(10)

In [ ]:
# map labels to common names
names_dict = dict(zip(labels_info.index, labels_info["Common Name"]))
print(names_dict)

In [ ]:
import numpy as np
from tensorflow import keras

data_generator_with_aug = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                       horizontal_flip=True,
                                                                       width_shift_range=0.2,
                                                                       height_shift_range=0.2,
                                                                       rotation_range=30)

data_generator_no_aug = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = data_generator_with_aug.flow_from_directory('../input/10-monkey-species/training/training',
                                                              target_size=(min_width, min_height),
                                                              batch_size=batch_size,
                                                              shuffle=True,
                                                              class_mode='categorical')
validation_generator = data_generator_no_aug.flow_from_directory('../input/10-monkey-species/validation/validation',
                                                                 target_size=(min_width, min_height),
                                                                 batch_size=batch_size,
                                                                 shuffle=False,
                                                                 class_mode='categorical')

training a model from scratch
- is super slow (most likely reading and transforming the images is a big bottleneck)

In [ ]:
model_from_scratch = keras.Sequential(
    [
        keras.Input(shape=(min_width, min_height, 3)),
        keras.layers.Conv2D(32,kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(64,kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation='softmax')
    ]
)

model_from_scratch.compile(optimizer='rmsprop',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

model_from_scratch.summary()

history = model_from_scratch.fit_generator(train_generator,
                                 epochs=200,
                                 validation_data=validation_generator)

training a model based on xception

In [ ]:
model_xception = keras.Sequential(
    [
        keras.Input(shape=(min_width, min_height, 3)),
        keras.applications.Xception(weights="imagenet", include_top=False),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        #keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation='softmax')
    ]
)

model_xception.compile(optimizer='rmsprop',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

model_xception.summary()

In [ ]:
history = model_xception.fit_generator(train_generator,
                                 epochs=35,
                                 validation_data=validation_generator)

In [ ]:
model_from_scratch.save('xception') 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame

acc = history.history['accuracy']
df_acc = DataFrame (acc, columns=['acc'])
df_acc['val_acc'] = history.history['val_accuracy']
df_acc.index += 1

loss = history.history['loss']
df_loss = DataFrame (loss, columns=['loss'])
df_loss['val_loss'] = history.history['val_loss']
df_loss.index += 1

plt.figure(figsize=(14,6))
plt.title("accuracy of the model")
plt.xlabel("epochs")
plt.ylabel("accuracy")
sns.lineplot(data=df_acc)

plt.figure(figsize=(14,6))
plt.title("loss of the model")
plt.xlabel("epochs")
plt.ylabel("loss")
sns.lineplot(data=df_loss)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict the values from the validation datasetand convert them to one hot encoded vectors
y_pred = np.argmax(model_from_scratch.predict_generator(validation_generator, validation_generator.samples), axis = 1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true = validation_generator.classes, y_pred = y_pred)
# plot the confusion matrix
#plot_confusion_matrix(confusion_mtx, normalize=True, target_names=labels)
sns.heatmap(data=confusion_mtx, annot=True, cmap='rocket_r')